In [1]:
%matplotlib inline
import cantera as ct
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import pandas as pd
import csv
import matplotlib as mpl
import csv
mpl.rcParams['font.family'] = "Times New Roman"
mpl.rcParams['figure.dpi'] = 300

In [2]:
# Create a class for solving TPD simulation
class reactor_sol:
  def __init__(self):   
    self.gas_mole_fracs=[]
    self.surf_fracs111=[]
    self.surf_fracs211=[]
    self.surf_fracs100=[]
    self.surf_fracs110=[]
    self.surf_fracsEdge1=[]
    self.surf_fracsEdge2=[]
    self.surf_fracsEdge3=[]
    self.surf_fracsEdge4=[]
    self.Ti=[]

In [3]:
def reactorCalc(n, covdep, facetName= None, sensVar= None, val= None, ind= None):

    #Some parameters
    T=383                                   # Initial temperature in K
    p=1                                     # pressure in bar
    volume = 0.1e-6                         # volume of the catalyst zone
    beta=20.0                               # temperature ramp in K/min
    m_cat=0.040                             # catalyst mass in g
    cat_area = 10.4                         # m2/g 
    facet_fraction = [0.6923, 0.0439, 0.211, 0.0528, 0.0, 0.0] # [111, 211, 100, 110, edge12, edge23]
    sccm = 50*1e-6/60                   # Flow rate m3/s
    
    # input file containing the surface reaction mechanism
    if(covdep):
        yaml_file = 'multifacet_covdep.yaml'   # Simulation with coverage dependent thermodynamics
    else:
        yaml_file = 'multifacet_noCovdep.yaml' # Default
    
    # Flag for Edge
    edge_present = n
    print(edge_present)

    # import the gas model and set the initial conditions
    gas = ct.Solution(yaml_file, 'gas')
    gas.TPX = T, p*ct.one_atm, 'Ar:1, CO2:0.0, CO:0'

    # import the (111) surface
    surf111 = ct.Interface(yaml_file,'surface1', [gas])
    surf111.TP = T, p*ct.one_atm
    surf111.coverages = {'site(111)':1, 'O(111)':0, 'OC(111)':0 , 'CO2(111)':0}
    
    # import the (211) surface
    surf211 = ct.Interface(yaml_file,'surface2', [gas])
    surf211.TP = T, p*ct.one_atm
    surf211.coverages = {'site(211)':1, 'O(211)':0, 'OC(211)':0 , 'CO2(211)':0}

    #import the (100) surface
    surf100 = ct.Interface(yaml_file,'surface3', [gas])
    surf100.TP = T, p*ct.one_atm
    surf100.coverages = {'site(100)':1, 'O(100)':0, 'OC(100)':0 , 'CO2(100)':0}
    
    #import the (110) surface
    surf110 = ct.Interface(yaml_file,'surface4', [gas])
    surf110.TP = T, p*ct.one_atm
    surf110.coverages = {'site(110)':1, 'O(110)':0, 'OC(110)':0 , 'CO2(110)':0}
    
    saveFileName = "noEdge_"
    # Import Edge
    if(edge_present):
        edge1 = ct.Interface(yaml_file,'edge12', [gas, surf111, surf211])
        edge1.TP = T, p*ct.one_atm
        edge2 = ct.Interface(yaml_file,'edge23', [gas, surf100, surf211])
        edge2.TP = T, p*ct.one_atm
        edge3 = ct.Interface(yaml_file,'edge14', [gas, surf111, surf110])
        edge3.TP = T, p*ct.one_atm
        edge4 = ct.Interface(yaml_file,'edge34', [gas, surf110, surf100])
        edge4.TP = T, p*ct.one_atm
        # [111, 211, 100, 110, edge12, edge23, edge 14, edge34]
        facet_fraction = [0.6923, 0.0439, 0.211, 0.0528, 0.0439, 0.0439, 0.0528, 0.0528] 
#         facet_fraction = [0.6923, 0.0439, 0.211, 0.0528, 0.0439, 0.0, 0.0, 0.0] 
            
        # Normalize
        facet_fraction = facet_fraction/np.sum(facet_fraction)
        saveFileName = "edge_"
    
#     # Sensitivity analysis
#     if(facetName):
#         if(facetName == "surf100"):
#             facet = surf100
#         elif(facetName == "surf111"):
#             facet = surf111
#         elif(facetName == "surf110"):
#             facet = surf110
#         elif(facetName == "surf211"):
#             facet = surf211
#         elif(facetName == "edge12"):
#             facet = edge1
#         elif(facetName == "edge23"):
#             facet = edge2
#         elif(facetName == "edge14"):
#             facet = edge3
#         elif(facetName == "edge34"):
#             facet = edge4
#         else:
#             Print("Error: Invalid surface")  
    
#     if(sensVar == "rate"):       
#         if(ind < facet.n_reactions):
#             # reset all multipliers
#             facet.set_multiplier(1.0)
#             #perturb each reaction by dk
#             print("ind = ", ind, val)
#             facet.set_multiplier(val, ind)
#             #print(facet.reaction_equations()[ind])    
    
#     # get the species 
#     s=facet.species(ind)
#     print(s)
#     # call the original coefficients of the NASA polynomial
#     original_coeffs = s.thermo.coeffs
        
#     if(sensVar == "thermoChemistry"):
#         #create an empty array for the perturbation
#         perturbed_coeffs = np.ones_like(original_coeffs)
#         #fill the array initially with the original coeffs
#         perturbed_coeffs[:] = original_coeffs
#         #perturb the enthalpy of the adsorbate for the low and high temperature polynomial
#         perturbed_coeffs[6] = original_coeffs[6] - val
#         #This is not a mistake, both values have to be perturbed by exactly the same value, otherwise discontinuity
#         perturbed_coeffs[13] = original_coeffs[13] - val
         
#         #fit a new NASA polynomial through the data and update the thermo database
#         s.thermo = ct.NasaPoly2(100.000, 5000.000, ct.one_atm, perturbed_coeffs)
#         print("ind =", ind)
#         facet.modify_species(ind, s)   
        
    # Reactor simulation
    sol = reactor_sol()
    
    # Get indices of the gas and surface species,assuming all surfaces have the same order of species
    sol.i_co2=gas.species_index('CO2')
    sol.i_o2=gas.species_index('O2')
    sol.i_co=gas.species_index('CO')
    sol.i_surf_Ni=surf111.species_index('site(111)')
    sol.i_surf_CO=surf111.species_index('OC(111)')
    sol.i_surf_O=surf111.species_index('O(111)')
    sol.i_surf_C=surf111.species_index('C(111)')
    sol.i_surf_CO2=surf111.species_index('CO2(111)')  
    
    # Cantera setup
    upstream = ct.Reservoir(gas, name='upstream')
    downstream = ct.Reservoir(gas, name='downstream')
    
    r=ct.IdealGasReactor(gas, energy='off')
    r.volume=volume
    rsurf111=ct.ReactorSurface(surf111, r, A=facet_fraction[0]*cat_area*m_cat)
    rsurf211=ct.ReactorSurface(surf211, r, A=facet_fraction[1]*cat_area*m_cat)
    rsurf100=ct.ReactorSurface(surf100, r, A=facet_fraction[2]*cat_area*m_cat)
    rsurf110=ct.ReactorSurface(surf110, r, A=facet_fraction[3]*cat_area*m_cat)
    
    if(edge_present):
        rsurfEdge1 = ct.ReactorSurface(edge1, r, A=facet_fraction[4]*cat_area*m_cat)
        rsurfEdge2 = ct.ReactorSurface(edge2, r, A=facet_fraction[5]*cat_area*m_cat)
        rsurfEdge3 = ct.ReactorSurface(edge3, r, A=facet_fraction[6]*cat_area*m_cat)
        rsurfEdge4 = ct.ReactorSurface(edge4, r, A=facet_fraction[7]*cat_area*m_cat)
        
    # This is important since it adjusts the flow rates, 303 K is the temperature of the gas cylinders
    vdot = sccm  * ((ct.one_atm / gas.P) * ( 303 / 273.15)) # m^3/s 
    mass_flow_rate =  vdot* gas.density
    mflow = ct.MassFlowController(upstream, r, mdot=mass_flow_rate)
    v = ct.PressureController(r, downstream, primary=mflow, K=1e-9)
    
    sim=ct.ReactorNet([r])

    ##set relative and absolute tolerances on the simulation
    sim.rtol = 1.0e-04
    sim.atol = 1.0e-10
    sim.max_err_test_fails = 10000
#     sim.max_steps = 1000000
    rxn_time=np.linspace(0,6900,6900)
    
    sol.gas_mole_fracs=np.zeros([gas.n_species, len(rxn_time)])
    sol.surf_fracs111=np.zeros([surf111.n_species, len(rxn_time)])
    sol.surf_fracs211=np.zeros([surf211.n_species, len(rxn_time)])
    sol.surf_fracs100=np.zeros([surf100.n_species, len(rxn_time)])
    sol.surf_fracs110=np.zeros([surf110.n_species, len(rxn_time)])
    if(edge_present):
        sol.surf_fracsEdge1=np.zeros([edge1.n_species, len(rxn_time)])
        sol.surf_fracsEdge2=np.zeros([edge2.n_species, len(rxn_time)])
        sol.surf_fracsEdge3=np.zeros([edge3.n_species, len(rxn_time)])
        sol.surf_fracsEdge4=np.zeros([edge4.n_species, len(rxn_time)])

    sol.Ti=np.zeros(len(rxn_time))
    sim.max_time_step = 1e-4
    
    for i in range(len(rxn_time)):
        time=rxn_time[i]
        print(time)
        if time < 10:
            sim.max_time_step = 2
            upstream.thermo.TPX=T,p*ct.one_atm, 'Ar:1, He:0, CO2:0'
            upstream.syncState()
        elif (time > 10) and (time < 3610):  
#             print(time)
            sim.max_time_step = 1e-3
            upstream.thermo.TPX=T,p*ct.one_atm, 'Ar:0, He:0.9013, CO2:0.0987'
            upstream.syncState()
        elif (time > 3610) and (time < 3970): 
            sim.max_time_step = 10
            upstream.thermo.TPX=T,p*ct.one_atm, 'Ar:1, He:0, CO2:0'
            upstream.syncState()
        elif (time > 3970) and (time < 5410):
            sim.max_time_step = 10
            upstream.thermo.TPX=323,p*ct.one_atm, 'Ar:1, He:0, CO2:0'   
            upstream.syncState()
            r.thermo.TP=323,ct.one_atm
            r.syncState()
        else:
            sim.max_time_step = 1e-2  
            r.thermo.TP=323+beta/60.0*(rxn_time[i]-5410),ct.one_atm
            r.syncState()
        
        sim.reinitialize
        sim.advance(time)
        sol.Ti[i]=r.thermo.T
        sol.gas_mole_fracs[:,i]=gas.X
        sol.surf_fracs111[:,i]=rsurf111.kinetics.X 
        sol.surf_fracs211[:,i]=rsurf211.kinetics.X 
        sol.surf_fracs100[:,i]=rsurf100.kinetics.X 
        sol.surf_fracs110[:,i]=rsurf110.kinetics.X
        if(edge_present):
            sol.surf_fracsEdge1[:,i]=rsurfEdge1.kinetics.X
            sol.surf_fracsEdge2[:,i]=rsurfEdge2.kinetics.X
            sol.surf_fracsEdge3[:,i]=rsurfEdge3.kinetics.X
            sol.surf_fracsEdge4[:,i]=rsurfEdge4.kinetics.X
#             print(np.sum(rsurf111.kinetics.X))
#             print(np.sum(rsurf211.kinetics.X))
#             print(np.sum(rsurf100.kinetics.X))
#             print(np.sum(rsurf110.kinetics.X))
            
    # Get the maximum and the position of the maximum of the TPD profile
    ind0=5410

    sol.max_height_CO2=np.max(sol.gas_mole_fracs[sol.i_co2, ind0:])*1e6
    sol.max_Temp_CO2=sol.Ti[np.argmax(sol.gas_mole_fracs[sol.i_co2,ind0:])+ind0]

    sol.max_height_CO=np.max(sol.gas_mole_fracs[sol.i_co, ind0:])*1e6
    sol.max_Temp_CO=sol.Ti[np.argmax(sol.gas_mole_fracs[sol.i_co,ind0:])+ind0]
        
    print(sol.max_height_CO2)
    
#     if(sensVar == "rate"):       
#         if(ind < facet.n_reactions):
#             # reset all multipliers
#             facet.set_multiplier(1.0)
    
#     if(sensVar == "thermoChemistry"):
#         #restore the original NASA polynomial and perturb the next species (only one at a time)
#         s.thermo = ct.NasaPoly2(100.000, 5000.000, ct.one_atm, original_coeffs)
#         facet.modify_species(ind, s)
        
    return sol

In [4]:
# Plot gas-phase species
def plot_gasSpecies(sol0, sol1):
    plt.rcParams['figure.figsize']=(6,3)
    plt.rcParams['axes.linewidth'] = 1.5
    gs = gridspec.GridSpec(1, 1)
    plt.rcParams.update({'font.size': 18})
    #gs.update(wspace = 0.4)
    ax0 = plt.subplot(gs[0])
    idx = 5410
    ax0.plot(sol0.Ti[idx:], 1e6*sol0.gas_mole_fracs[sol0.i_co,idx:], label='CO$^0$',color='k', linestyle='dashed', linewidth=3)
    ax0.plot(sol1.Ti[idx:], 1e6*sol1.gas_mole_fracs[sol1.i_co,idx:], label='CO$^{Diff}$',color='k', linewidth=3)
    ax0.plot(sol0.Ti[idx:], 1e6*sol0.gas_mole_fracs[sol0.i_co2,idx:], label='CO$_2^0$',color='b', linestyle='dashed', linewidth=3)
    ax0.plot(sol1.Ti[idx:], 1e6*sol1.gas_mole_fracs[sol1.i_co2,idx:], label='CO$_2^{Diff}$',color='b', linewidth=3)
    ax0.set_xlabel('Temperature [K]')
    ax0.set_xlim([323, 800])
    #ax0.set_ylim([0, 1000])
    ax0.set_ylabel('Species concentration [ppm]')
    ax0.legend()
    plt.savefig('gasPhase.png', dpi=300)
    plt.savefig('gasPhase.svg', dpi=1000)
    #plt.savefig('gasPhase.pdf', dpi=1000)
    plt.show()

In [5]:
# Plot surface species

def plot_surfSpecies(sol0, sol1, arr1, arr2, filename):
    plt.rcParams['figure.figsize']=(6,3)
    plt.rcParams['axes.linewidth'] = 1.5
    gs = gridspec.GridSpec(1, 1)
    plt.rcParams.update({'font.size': 18})
    #gs.update(wspace = 0.4)
    i_surf_CO = sol0.i_surf_CO
    i_surf_O = sol0.i_surf_O
    i_surf_CO2 = sol0.i_surf_CO2
    i_surf_Ni = sol0.i_surf_Ni
    ax0 = plt.subplot(gs[0])
    idx = 5410
    ax0.plot(sol0.Ti[idx:], arr1[sol0.i_surf_CO,idx:], label='$\mathrm{CO(s)}^0$', linestyle='dashed', color='k', linewidth=3)
    ax0.plot(sol0.Ti[idx:], arr1[sol0.i_surf_O,idx:], label='$\mathrm{O(s)}^0$', linestyle='dashed', color='b', linewidth=3)
    #ax0.plot(sol0.Ti[idx:], arr1[sol0.i_surf_CO2,idx:], label='$\mathrm{CO2(s)}^0$', linestyle='dashed', color='r', linewidth=3)
    ax0.plot(sol0.Ti[idx:], arr1[sol0.i_surf_Ni,idx:], label='$\mathrm{Ni(s)}^0$', linestyle='dashed', color='m', linewidth=3)#ax0.plot(sol1.Ti[100:], sol0.surf_fracs111[i_surf_CO,100:], label='$\mathrm{CO(111)}$', color='k', linewidth=3)
    ax0.plot(sol1.Ti[idx:], arr2[sol1.i_surf_CO,idx:], label='$\mathrm{CO(s)}^{Diff}$', color='k', linewidth=3)
    ax0.plot(sol1.Ti[idx:], arr2[sol1.i_surf_O,idx:], label='$\mathrm{O(s)}^{Diff}$', color='b', linewidth=3)
    #ax0.plot(sol1.Ti[idx:], arr2[sol1.i_surf_CO2,idx:], label='$\mathrm{CO2(s)}^{Diff}$', color='r', linewidth=3)
    ax0.plot(sol1.Ti[idx:], arr2[sol1.i_surf_Ni,idx:], label='$\mathrm{Ni(s)}^{Diff}$', color='m', linewidth=3)
    ax0.set_xlabel('Temperature [K]')
    ax0.set_xlim([323, 800])
    #ax0.set_ylim([0, 1200])
    ax0.set_ylabel('Species coverages []')
    ax0.legend()#(bbox_to_anchor=(0, 0.85, 1.4, 0), ncol = 1, loc='upper right')
    #plt.title('A_edge = '+str(area))
    plt.savefig(filename + ".png", dpi=300)
    plt.savefig(filename + ".svg", dpi=1000)
    #plt.savefig(filename + ".pdf", dpi=1000)
    plt.show()

In [ ]:
# import csv
# def write2CSV(filename, surfname, data, fields = None):
    
#     with open(filename, 'a', newline='') as csvfile:
#         # creating a csv writer object 
#         csvwriter = csv.writer(csvfile)
        
#         # writing the data 
#         csvwriter.writerow("\n")
#         csvwriter.writerow(surfname)
#         csvwriter.writerow(fields)
#         csvwriter.writerows(data)
#         csvwriter.writerow("\n")

In [ ]:
# pd.DataFrame(np.transpose(sol0.Ti)).to_csv("Temperature.csv", index=False, header=False) 
# data0 = np.concatenate((sol0.gas_mole_fracs, sol0.surf_fracs100, sol0.surf_fracs110, sol0.surf_fracs111, sol0.surf_fracs211), axis = 0)
# data0 = np.transpose(data0)
# pd.DataFrame(data0).to_csv("sol_noDiffusion.csv", index=False, header=False)
# data1 = np.concatenate((sol1.gas_mole_fracs, sol1.surf_fracs100, sol1.surf_fracs110, sol1.surf_fracs111, sol1.surf_fracs211), axis = 0)
# data1 = np.transpose(data1)
# pd.DataFrame(data1).to_csv("sol_diffusion.csv", index=False, header=False)

In [ ]:
###################################### SOLVE ############################################


####### With no coverage dependence

# # Edge + covdep
sol11 = reactorCalc(1,1)
print(sol11.max_height_CO2, sol11.max_height_CO, sol11.max_Temp_CO2, sol11.max_Temp_CO)

# # No edge + covdep
sol01 = reactorCalc(0,1)
print(sol01.max_height_CO2, sol01.max_height_CO, sol01.max_Temp_CO2, sol01.max_Temp_CO)

######  With coverage dependence

# # # Edge + no covdep
# sol10 = reactorCalc(1,0)
# print(sol10.max_height_CO2, sol10.max_height_CO, sol10.max_Temp_CO2, sol10.max_Temp_CO)

# # # No edge + no covdep
# sol00 = reactorCalc(0,0)
# print(sol00.max_height_CO2, sol00.max_height_CO, sol00.max_Temp_CO2, sol00.max_Temp_CO)

In [ ]:
# # Write solution data 
# pd.DataFrame(np.transpose(sol01.Ti)).to_csv("Temperature.csv", index=False, header=False) 

# data01 = np.concatenate((sol01.gas_mole_fracs, sol01.surf_fracs100, sol01.surf_fracs110, sol01.surf_fracs111, sol01.surf_fracs211), axis = 0)
# data01 = np.transpose(data01)
# pd.DataFrame(data01).to_csv("sol_noDiffusion_covdep.csv", index=False, header=False)

# data11 = np.concatenate((sol11.gas_mole_fracs, sol11.surf_fracs100, sol11.surf_fracs110, sol11.surf_fracs111, sol11.surf_fracs211), axis = 0)
# data11 = np.transpose(data11)
# pd.DataFrame(data11).to_csv("sol_diffusion_covdep.csv", index=False, header=False)

# data00 = np.concatenate((sol00.gas_mole_fracs, sol00.surf_fracs100, sol00.surf_fracs110, sol00.surf_fracs111, sol00.surf_fracs211), axis = 0)
# data00 = np.transpose(data00)
# pd.DataFrame(data00).to_csv("sol_noDiffusion_noCovdep.csv", index=False, header=False)

# data10 = np.concatenate((sol10.gas_mole_fracs, sol10.surf_fracs100, sol10.surf_fracs110, sol10.surf_fracs111, sol10.surf_fracs211), axis = 0)
# data10 = np.transpose(data10)
# pd.DataFrame(data10).to_csv("sol_diffusion_noCovdep.csv", index=False, header=False)

In [ ]:
##################################### PLOT ############################################

# Plot surface species (with covdep)
plot_gasSpecies(sol01, sol11)
plot_surfSpecies(sol01, sol11, sol01.surf_fracs100, sol11.surf_fracs100, 'surf_coverages100')
plot_surfSpecies(sol01, sol11, sol01.surf_fracs211, sol11.surf_fracs211, 'surf_coverages211')
plot_surfSpecies(sol01, sol11, sol01.surf_fracs110, sol11.surf_fracs110, 'surf_coverages110')
plot_surfSpecies(sol01, sol11, sol01.surf_fracs111, sol11.surf_fracs111, 'surf_coverages111')

# Plot surface species (with no covdep)
plot_gasSpecies(sol00, sol10)
plot_surfSpecies(sol00, sol10, sol00.surf_fracs100, sol10.surf_fracs100, 'surf_coverages100')
plot_surfSpecies(sol00, sol10, sol00.surf_fracs211, sol10.surf_fracs211, 'surf_coverages211')
plot_surfSpecies(sol00, sol10, sol00.surf_fracs110, sol10.surf_fracs110, 'surf_coverages110')
plot_surfSpecies(sol00, sol10, sol00.surf_fracs111, sol10.surf_fracs111, 'surf_coverages111')

In [ ]:
# Comparison with the experimental data
import pandas as pd
filename = 'C:/Users/gkoge/Jupyter notebooks/multifacet/manuscript/TPD_experiments.csv'
#print(filename)
data = pd.read_csv(filename).values
T_expt = data[:,0] + 273.15
co2_expt = data[:,1]
co_expt = data[:,2]

In [ ]:
# Plot
plt.rcParams['figure.figsize']=(12,6)
gs = gridspec.GridSpec(1, 1)
plt.rcParams.update({'font.size': 24})
gs.update(wspace = 0.4)
ax0 = plt.subplot(gs[0])
idx = 5410
ax0.plot(T_expt, 1e4*co_expt, label='CO$^{expt}$',color='k', linestyle='dashed', linewidth=3)
ax0.plot(sol0.Ti[idx:], 1e6*sol0.gas_mole_fracs[sol0.i_co,idx:], label='CO$^{0}$',color='k', linestyle='dashdot', linewidth=3)
ax0.plot(sol1.Ti[idx:], 1e6*sol1.gas_mole_fracs[sol1.i_co,idx:], label='CO$^{diff}$',color='k', linewidth=3)
ax0.plot(T_expt, 1e4*co2_expt, label='CO$_2^{expt}$',color='b', linestyle='dashed', linewidth=3)
ax0.plot(sol0.Ti[idx:], 1e6*sol0.gas_mole_fracs[sol0.i_co2,idx:], label='CO$_2^{0}$',color='b', linestyle='dashdot', linewidth=3)
ax0.plot(sol1.Ti[idx:], 1e6*sol1.gas_mole_fracs[sol1.i_co2,idx:], label='CO$_2^{diff}$',color='b', linewidth=3)
ax0.set_xlabel('Temperature [K]')
ax0.set_xlim([323, 800])
#ax0.set_ylim([0, 1200])
ax0.set_ylabel('Species concentration [ppm]')
ax0.legend()
plt.savefig('gasPhase_expt.png', dpi=300)
plt.savefig('gasPhase_expt.svg', dpi=1000)
#plt.savefig('gasPhase_expt.pdf', dpi=1000)
plt.show()

In [ ]:
# Plot
plt.rcParams['figure.figsize']=(6,8)
gs = gridspec.GridSpec(1, 1)
plt.rcParams.update({'font.size': 18})
gs.update(wspace = 0.4)
ax0 = plt.subplot(gs[0])
idx = 5410
ax0.plot(T_expt, 1e4*co_expt, label='CO$^{expt}$',color='k', linestyle='none', marker='+', markevery=25)
ax0.plot(sol0.Ti[idx:], 1e6*sol0.gas_mole_fracs[sol0.i_co,idx:], label='CO$^{0}$',color='k', linestyle='dashed', linewidth=3)
ax0.plot(sol1.Ti[idx:], 1e6*sol1.gas_mole_fracs[sol1.i_co,idx:], label='CO',color='k', linewidth=3)
ax0.plot(T_expt, 1e4*co2_expt, label='CO$_2^{expt}$',color='b', linestyle='none', marker='+', markevery=25)
ax0.plot(sol0.Ti[idx:], 1e6*sol0.gas_mole_fracs[sol0.i_co2,idx:], label='CO$_2^{0}$',color='b', linestyle='dashed', linewidth=3)
ax0.plot(sol1.Ti[idx:], 1e6*sol1.gas_mole_fracs[sol1.i_co2,idx:], label='CO$_2$',color='b', linewidth=3)
ax0.set_xlabel('Temperature [K]')
ax0.set_xlim([323, 800])
#ax0.set_ylim([0, 1200])
ax0.set_ylabel('Species concentration [ppm]')
#ax0.legend()
plt.savefig('gasPhase_expt.png', dpi=300)
plt.savefig('gasPhase_expt.svg', dpi=1000)
#plt.savefig('gasPhase_expt.pdf', dpi=1000)
plt.show()

In [ ]:
# import csv
# def write2CSV(filename, surfname, data, fields = None):
    
#     with open(filename, 'a', newline='') as csvfile:
#         # creating a csv writer object 
#         csvwriter = csv.writer(csvfile)
        
#         # writing the data 
#         csvwriter.writerow("\n")
#         csvwriter.writerow(surfname)
#         csvwriter.writerow(fields)
#         csvwriter.writerows(data)
#         csvwriter.writerow("\n")

In [ ]:
# # Function to perform the sensitivity analysis with respect to the rates
# def sens_rate(n, covdep, filename, facetName, nReactions):
#     heights_CO2,heights_CO=[],[]
#     temps_CO2,temps_CO=[],[]
    
#     dk=1.0e-2
    
#     #cycle through each reaction",surf.n_reactions
#     for m in range(nReactions):
        
#         # Run the simulation without the edge
#         sol_m = reactorCalc(n, covdep, facetName, "rate", 1+dk, m)
        
#         heights_CO2.append(sol_m.max_height_CO2)
#         temps_CO2.append(sol_m.max_Temp_CO2)

#         heights_CO.append(sol_m.max_height_CO)
#         temps_CO.append(sol_m.max_Temp_CO)
        
#     # Save the data
#     dataArray= np.zeros((4, nReactions))
#     dataArray[0][:] = heights_CO2
#     dataArray[1][:] = temps_CO2
#     dataArray[2][:] = heights_CO
#     dataArray[3][:] = temps_CO

#     # Take transpose
#     dataArray = np.transpose(dataArray)    
    
#     # Write to CSV
#     # field 
#     fields = ['Heights_CO2', 'Temps_CO2', 'Heights_CO', 'Temp_CO2'] 
#     write2CSV(filename, facetName, dataArray, fields) 
#     print(heights_CO2, heights_CO, temps_CO2, temps_CO)
                
#     return heights_CO2, heights_CO, temps_CO2, temps_CO            

In [ ]:
# # Function to perform the sensitivity analysis with respect to the rates
# def sens_thermo(n, covdep, filename, facetName, nSpecies):
    
#     fields = ['Heights_CO2', 'Temps_CO2', 'Heights_CO', 'Temp_CO2']
#     #with open(filename, 'a', newline='') as csvfile:
#     #    # creating a csv writer object 
#     #    csvwriter = csv.writer(csvfile)        
#     #    # writing the fields 
#     #    csvwriter.writerow(fields)
        
#     heights_CO2,heights_CO=[],[]
#     temps_CO2,temps_CO=[],[]
    
#     dH=1.0e-2
#     dk = dH*1000 / 8.314  # for the thermo loop, 'dk' is in fact (delta H / R)
    
#     #cycle through each reaction",surf.n_reactions
#     for m in range(nSpecies):
        
#         # Run the simulation without the edge
#         sol_m = reactorCalc(n, covdep, facetName, "thermoChemistry", 1+dk, m)
        
#         heights_CO2.append(sol_m.max_height_CO2)
#         temps_CO2.append(sol_m.max_Temp_CO2)

#         heights_CO.append(sol_m.max_height_CO)
#         temps_CO.append(sol_m.max_Temp_CO)
        
#     # Save the data
#     dataArray= np.zeros((4, nSpecies))
#     dataArray[0][:] = heights_CO2
#     dataArray[1][:] = temps_CO2
#     dataArray[2][:] = heights_CO
#     dataArray[3][:] = temps_CO

#     # Take transpose
#     dataArray = np.transpose(dataArray)    
    
#     # Write to CSV
#     write2CSV(filename, facetName, dataArray, fields)
        
#     return heights_CO2, heights_CO, temps_CO2, temps_CO            

In [ ]:
# Sensitivity analysis based on chemical rates
# filename = "rate_diff_covdep.csv"
# [heights_CO2, heights_CO, temps_CO2, temps_CO] = sens_rate(1, 1, filename, "surf111", 5)
# [heights_CO2, heights_CO, temps_CO2, temps_CO] = sens_rate(1, 1, filename,"surf100", 5)
# [heights_CO2, heights_CO, temps_CO2, temps_CO] = sens_rate(1, 1, filename,"surf211", 5)
# [heights_CO2, heights_CO, temps_CO2, temps_CO] = sens_rate(1, 1, filename,"surf110", 5)
# [heights_CO2, heights_CO, temps_CO2, temps_CO] = sens_rate(1, 1, filename,"edge12", 3)
# [heights_CO2, heights_CO, temps_CO2, temps_CO] = sens_rate(1, 1, filename,"edge23", 3)
# [heights_CO2, heights_CO, temps_CO2, temps_CO] = sens_rate(1, 1, filename,"edge14", 3)
# [heights_CO2, heights_CO, temps_CO2, temps_CO] = sens_rate(1, 1, filename,"edge34", 3)

# filename = "rate_diff_nocovdep.csv"
# [heights_CO2, heights_CO, temps_CO2, temps_CO] = sens_rate(1, 0, filename, "surf111", 5)
# [heights_CO2, heights_CO, temps_CO2, temps_CO] = sens_rate(1, 0, filename,"surf100", 5)
# [heights_CO2, heights_CO, temps_CO2, temps_CO] = sens_rate(1, 0, filename,"surf211", 5)
# [heights_CO2, heights_CO, temps_CO2, temps_CO] = sens_rate(1, 0, filename,"surf110", 5)
# [heights_CO2, heights_CO, temps_CO2, temps_CO] = sens_rate(1, 0, filename,"edge12", 3)
# [heights_CO2, heights_CO, temps_CO2, temps_CO] = sens_rate(1, 0, filename,"edge23", 3)
# [heights_CO2, heights_CO, temps_CO2, temps_CO] = sens_rate(1, 0, filename,"edge14", 3)
# [heights_CO2, heights_CO, temps_CO2, temps_CO] = sens_rate(1, 0, filename,"edge34", 3)

In [ ]:
# # Sensitivity analysis based on thermochemistry
# # filename = "sensitivity_thermo_diff_covdep.csv"

# # [heights_CO2, heights_CO, temps_CO2, temps_CO] = sens_thermo(1, 1, filename, "surf111", 5)
# # [heights_CO2, heights_CO, temps_CO2, temps_CO] = sens_thermo(1, 1, filename,"surf100", 5)
# # [heights_CO2, heights_CO, temps_CO2, temps_CO] = sens_thermo(1, 1, filename,"surf211", 5)
# # [heights_CO2, heights_CO, temps_CO2, temps_CO] = sens_thermo(1, 1, filename,"surf110", 5)
# # [heights_CO2, heights_CO, temps_CO2, temps_CO] = sens_thermo(1, 1, filename,"edge12", 1)
# # [heights_CO2, heights_CO, temps_CO2, temps_CO] = sens_thermo(1, 1, filename,"edge23", 1)
# # [heights_CO2, heights_CO, temps_CO2, temps_CO] = sens_thermo(1, 1, filename,"edge14", 1)
# # [heights_CO2, heights_CO, temps_CO2, temps_CO] = sens_thermo(1, 1, filename,"edge34", 1)

# # Sensitivity analysis based on thermochemistry
# filename = "sensitivity_thermo_diff_nocovdep.csv"

# [heights_CO2, heights_CO, temps_CO2, temps_CO] = sens_thermo(1, 0, filename, "surf111", 5)
# [heights_CO2, heights_CO, temps_CO2, temps_CO] = sens_thermo(1, 0, filename,"surf100", 5)
# [heights_CO2, heights_CO, temps_CO2, temps_CO] = sens_thermo(1, 0, filename,"surf211", 5)
# [heights_CO2, heights_CO, temps_CO2, temps_CO] = sens_thermo(1, 0, filename,"surf110", 5)

In [ ]:
# import csv
# filename = 'temp.csv'
# mode = 'a'
# # Save the data
# dataArray= np.zeros((4, 5))
# dataArray[0][:] = [0,1,2,3,4]
# dataArray[1][:] = [10, 11 ,12, 13, 14 ]
# dataArray[2][:] = [0, 1, 2, 3, 5]
# dataArray[3][:] = [10, 11, 12, 13, 74 ]
# dataArray = np.transpose(dataArray)
# with open(filename, mode, newline='') as csvfile:
#     writer = csv.writer(csvfile)
#     writer.writerows(dataArray)